In [ ]:
import sys
import logging, os
from time import time
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib notebook

import ipyparallel as ipp

from bem import Electrodes, Sphere, Cylinder, Mesh, Grid, Configuration, Result
from bem.formats import stl
from bem.bemColors_lib.bemColors import bemColors
from helper_functions import helper_functions

: 

# `I. Read STL and rename electrodes` 
This part reads a colored model file `./SimpleTrap.stl`, and assigns a name for each color(thus for each electrode). The result is stored in `./SimpleTrap_mesh.vtk`.

Here we name different electrodes and refine the meshes. The size of mesh deternmines the accuracy in next step. You should set the mesh finer and finer until the result converges. Also, you can set the mesh finer and test the convergence.

## (1) color your electrodes (in fusion 360) and export stl
A trap always consists of different electrodes. Here we use different color to identify different electrodes(STL file stores color for each mesh). In fusion 360/Inventor, you can assign different appearance for different parts, of which we only cares about the color(because stl file only stores color), not texture. We recommend to use ../../bemCol_lib/bemCol.adsklib for different electrodes if you are usin Fusion 360. Same color for two electrodes means they are shorted. Then export to an stl file and move the file in the same folder as the mesh processing file. 

Below, "prefix" should be the name of the STL file. 
"a0" should be close to the characteristic distance.
"unit" is the unit for the STL file.

In [ ]:
# base file name for outputs and inputs is the script name
prefix = "htrap_overhang_v2"

a0 = 1e-3    # Distance from ion to electrode is 40 µm.
unit = 1e-3   # unit of the stl file is µm.

Some notes:
* Why use standard color? We define a set of standard color lib `../../../bemCol_lib/bemCol.adsklib` for two reasons: 1. when exporting to .stl file, Fusion 360 will compress 0-256 color(RGBA32) into 0-32 color(RGBA16), which means that some similar colors in fusion 360 will become same color after export to .stl file. Our standard color lib avoid this problem. 2.If you use .stl file exported from Fusion 360 DIRECTLY, naming the electrodes would be very convinient. If you use other apps such as Inventor or Meshlab to generate .stl file, you can ignore the second reason because color encoding are quite different in different apps



## (2) The formal rename of electrode. 
### (2.1) rename directly with color code
First read stl and drop all color codes. Be sure to put something random in rename argument to drop all color codes.

In [3]:
# load electrode faces from colored stl
# s_nta is intermediate processed stl file.
s_nta = stl.read_stl(open("%s.stl" % prefix, "rb"))
print("Import stl:",os.path.abspath("./"+prefix+".stl"),"\n")
print("Electrode colors (numbers):\n")
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=a0/unit,rename={1:"1"}))   
# after scale, unit is a0

Import stl: /home/sqip/Documents/github/bem/examples/SimpleTrap/htrap_overhang_v2.stl 

Electrode colors (numbers):

dropping 20083
dropping 28593
dropping 5345
dropping 13683
dropping 27949
dropping 13363
dropping 15669
dropping 28135
dropping 5997
dropping 5493
dropping 25969
dropping 1647
dropping 7275
dropping 19501
dropping 28463
dropping 10151
dropping 1399
dropping 15905
dropping 5941
dropping 10095
dropping 19943
dropping 9893
dropping 11517
dropping 5691
mesh.gather() not working properly, be sure to have valid input


Assign each electrode a string name instead of its color coding. Use the numbers you get above.  
`stl.stl_to_mesh()` prints normal vectors (different faces) in each electrode.

In [4]:
print(len(s_nta), type(s_nta),"\n")
# s_nta is a length 3 tuple. (normal, triangle, attribute) 
# Normal direction of each triangle, three vetice of triangles, coding number of colors.
print("Triangles:",len(s_nta[0]),"\nColors:",len(s_nta[2]),"\n")    # This isn't right.

# stl_to_mesh() only assigns names and does scaling, doing no triangulation to stl mesh.
# "scale=a0/unit" only scales dimensionless
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=a0/unit,
    rename={9495: "DC1", 
            17962: "DC3", 
            18994: "DC5",
            18869: "DC2", 
            20943: "RF", 
            18129: "DC4"
           }, quiet=False))    


3 <class 'tuple'> 

Triangles: 3850 
Colors: 3850 

dropping 20083
dropping 28593
dropping 5345
dropping 13683
dropping 27949
dropping 13363
dropping 15669
dropping 28135
dropping 5997
dropping 5493
dropping 25969
dropping 1647
dropping 7275
dropping 19501
dropping 28463
dropping 10151
dropping 1399
dropping 15905
dropping 5941
dropping 10095
dropping 19943
dropping 9893
dropping 11517
dropping 5691
mesh.gather() not working properly, be sure to have valid input


### (2.2) rename with BemColors

Below shows how to rename electrodes with `BemColors`

The code is fully commented out, because the SimpleTrap.stl file doesn't use Fusion 360 colors. But The basic

Firstly, we print all the colors appear in the stl file. If a color is of standard colors in `bemCol.adsklib` or named manually by function `bemCol.set_my_color()`, then the color will appear in a straightforward name, such as `bem0`,`bem1` ...... . Otherwise, the color will simply be named as `'_unkCol0'` , `'_unkCol1'`, ... 


In [5]:
ele_col = bemColors(np.array(list(set(s_nta[2]))),('fusion360','export_stl'))
ele_col.set_my_color(value = (178,178,178),cl_format = ('fusion360','export_stl','RGBA64'),name = 'self_defined')
ele_col.print_stl_colors()

COLORS in the stl:
['bem6']
['bem13']
['bem19']
['bem11']
['bem17']
['bem15']
['bem30']
['bem1']
['bem12']
['bem5']
['bem25']
['bem18']
['bem3']
['bem16']
['bem7']
['bem9']
['bem2']
['bem4']
['bem20']
['bem10']
['bem14']
['bem8']
['bem21']
['_unkColor0']
TOTAL COLORS:  24


Next, you need to assign a name for each color appeared above. With standard colors defined in `bemCol.adsklib`, the correspondence between color and electrode is clear, which make it easy for this step. You can also comment out some `ele_col.set_color_name()`, run all the codes again and observe the missing part in the printed figure, which corresponds to the electrode you comment out. The second method also serves as a double check. Besides, parameter in `stl.stl_to_mesh()` can be set as `quiet = False` to check whether the program reads planes correctly.

In [6]:
# # # assign a name for each color
# ele_col.color_electrode(color = 'bem1',name = 'DC1')
# ele_col.color_electrode(color = 'bem2',name = 'DC2')
# ele_col.color_electrode(color = 'bem3',name = 'DC3')
# ele_col.color_electrode(color = 'bem4',name = 'DC4')
# ele_col.color_electrode(color = 'bem5',name = 'DC5')
# ele_col.color_electrode(color = 'bem6',name = 'DC6')
# ele_col.color_electrode(color = 'bem7',name = 'DC7')
# ele_col.color_electrode(color = 'bem8',name = 'DC8')
# ele_col.color_electrode(color = 'bem9',name = 'DC9')
# ele_col.color_electrode(color = 'bem10',name = 'DC10')
# ele_col.color_electrode(color = 'bem11',name = 'DC11')
# ele_col.color_electrode(color = 'bem12',name = 'DC12')
# ele_col.color_electrode(color = 'bem13',name = 'DC13')
# ele_col.color_electrode(color = 'bem14',name = 'DC14')
# ele_col.color_electrode(color = 'bem15',name = 'DC15')
# ele_col.color_electrode(color = 'bem16',name = 'DC16')
# ele_col.color_electrode(color = 'bem17',name = 'DC17')
# ele_col.color_electrode(color = 'bem18',name = 'DC18')
# ele_col.color_electrode(color = 'bem19',name = 'DC19')
# ele_col.color_electrode(color = 'bem20',name = 'DC20')
ele_col.color_electrode(color = 'bem21',name = 'DC21')
ele_col.color_electrode(color = 'bem30',name = 'DC0')
ele_col.color_electrode(color = 'bem25',name = 'RF')

# print colors still with no name. These meshes will be neglected in the following codes
ele_col.drop_colors()

# read stl into mesh with electrode names
# unnamed meshes will not be imported at all
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=a0/unit,
    rename=ele_col.electrode_colors, quiet=True))

dropping color ['bem6']
dropping color ['bem13']
dropping color ['bem19']
dropping color ['bem11']
dropping color ['bem17']
dropping color ['bem15']
dropping color ['bem1']
dropping color ['bem12']
dropping color ['bem5']
dropping color ['bem18']
dropping color ['bem3']
dropping color ['bem16']
dropping color ['bem7']
dropping color ['bem9']
dropping color ['bem2']
dropping color ['bem4']
dropping color ['bem20']
dropping color ['bem10']
dropping color ['_unkColor0']
dropping color ['bem14']
dropping color ['bem8']
TOTAL COLORS DROPPED:  21
dropping 20083
dropping 5345
dropping 13683
dropping 27949
dropping 13363
dropping 15669
dropping 28135
dropping 5997
dropping 5493
dropping 25969
dropping 1647
dropping 7275
dropping 19501
dropping 28463
dropping 10151
dropping 1399
dropping 15905
dropping 5941
dropping 10095
dropping 19943
dropping 9893


# `II. Remesh with constraint` 

In this step, we generate triangle mesh with constraints. The meshes are 2-dimensional triangles on the surface of electrodes. The region enclosed by constraint shape can have finer mesh. Triangulation is done by `triangle` C library. Folowing variables are all in unit `mesh_unit` now.

Our remesh strategy consists of two steps of triangulation: 
1. global triangulation without constraint. This step eliminate some long and sharp triangles by combining and dividing, and obtains a coarse grain triangulated model.
2. local triangulation with constraint. This step refines each triangles in step 1, the triangle density is defined by `mesh.areas_from_constraints`

It is better to update triangle density gradually to accurately position the constraint boundary. Parameters in the below code block should be tuned specificly for different trap geometries.

Notes:
- mesh.triangulate function takes in a parameter opts, after some processing of the wrap function, opts changes a little and prints as final opts. These final opts finally passes into Fastlap triangulate package, of which parameters and usage can be found in this link (http://www.cs.cmu.edu/~quake/triangle.research.html).

In [7]:
# 1. Global triangulation
mesh.triangulate(opts="q20Q",new = False)

# 2. Local triangulation
# areas_from_constraints specifies sphere/cylinder with finer mesh inside it.
# "inside", "outside" set different mesh densities.

# first refine
mesh.areas_from_constraints(Sphere(center=np.array([0, 0, 1.]), radius=2, inside=2., outside=10))    # Sphere constraint
# mesh.areas_from_constraints(Cylinder(start = np.array([-1, 0, 0]), end = np.array([1, 0, 0]), radius = 2, inside=2, outside=10)) # Cyliner constraint
mesh.triangulate(opts="q20Q", new=False) # retriangulate quality and quiet with areas

# second refine
mesh.areas_from_constraints(Sphere(center=np.array([0, 0, 1.]), radius=2, inside=0.2, outside=10))    # Sphere constraint
# mesh.areas_from_constraints(Cylinder(start = np.array([-1, 0, 0]), end = np.array([1, 0, 0]), radius = 2, inside=0.2, outside=10)) # Cyliner constraint
mesh.triangulate(opts="q20Q", new=False) # retriangulate quality and quiet with areas


# 3. save base mesh to vtk
mesh.to_vtk(prefix)
print("Output vtk:",os.path.abspath("./"+prefix+".vtk"))    # output path

start triangulate DC0
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
finish triangulate DC0
start triangulate DC21
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
finish triangulate DC21
start triangulate RF
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
('final opts', 'q20Qzr')
finish triangulate RF
start triangulate DC0
('final opts', 'q20Qzra')
('final opts', 'q20Qzra')
('final opts', 'q20Qzra')
('final opts', 'q20Qzra')
('final opts', 'q20Qzra')
('final opts', 'q20Qzra')
('final opts', 'q20Qzra')
finish triangulate DC0
start triangulate DC21
('final opts', 'q20Qzra')
('final opts', 'q20Qzra')
('final opts', 'q20Qzra')
('final opts', 'q20Qzra')
('final opt

## Visualize in 3D to check mesh
Can be used to check whether electrode faces are colored properly, and whether electrodes are named properly.

Plot mesh using pyvista plotter in a separate window.
On macOS perhaps Linux as well, the window doesn't close properly after clicking close button, but the code can continue to run.

In [8]:
# Plot triangle meshes.
mesh.plot()

# `III. Main boundary element calculations`
## (1) Define job function

In `run_job` function, `job` is `Configuration` instance and `grid` is discretirized spatial grid (not the mesh). The general workflow (also the routine of BEM method) are:  
1. `solve_singularities()` solves charge distributions by iterative methods to make it consistent with one electrode at 1V and others at 0V (unit potentials). `adapt_mesh()` refines meshes adaptively to achieve certain precision while solving sigulartities.
2. Compute potentials on given grid points by `simulate()`, based on the charge distributions gotten previously.
3. Potential data of each unit potential are saved seperately to a `Result` instance, and also export to VTK files.
4. Return total accumulated charge per electrode in the end.

Major calculations calls `fastlap` C library which uses a pre-conditioned, adaptive, multipole-accelerated algorithm for solving Laplace problem. Two parameters control multipole acceleration.
+ num_mom, the number of multipole
+ num_lev, the number of levels in the hierarchical spatial decomposition.  
num_lev=1 means direct computation without multipole acceleration. See fastlap ug.pdf and README.rst.

In [9]:
# Define calculation function.
def run_job(args):
    # job is Configuration instance.
    job, grid, prefix = args
    # # refine twice adaptively with increasing number of triangles, min angle 25 deg.
    # job.adapt_mesh(triangles=4e2, opts="q25Q")
    # job.adapt_mesh(triangles=1e3, opts="q25Q")
    # solve for surface charges
    job.solve_singularities(num_mom=4, num_lev=3)
    # get potentials and fields
    result = job.simulate(grid, field=job.name=="RF", num_lev=2)    # For "RF", field=True computes the field.
    result.to_vtk(prefix)
    print("finished job %s" % job.name)
    return job.collect_charges()

## (2) Define grid

Create a grid in unit of scaled length `l`. Only choose the interested region (trap center) to save time.

In [19]:
# grid to evalute potential and fields atCreate a grid in unit of scaled length l. Only choose the interested region (trap center) to save time.
s = 5e-3
Lx, Ly, Lz = 100e-3, 100e-3, 100e-3    # in the unit of scaled length l
sx, sy, sz = s, s, s
# ni is grid point number, si is step size. Thus to fix size on i direction you need to fix ni*si.
nx, ny, nz = [2*np.ceil(L/2.0/ss).astype('int')+1 for ss, L in zip((sx, sy, sz), (Lx, Ly, Lz))]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(0, 0, 75e-3), step=(sx, sy, sz), shape=(nx, ny, nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("Grid origin/l:", grid.get_origin())

Size/l: 0.1 0.1 0.1
Step/l: 0.005 0.005 0.005
Shape (grid point numbers): 21 21 21
Grid origin/l: [-0.05  -0.05   0.025]


## (3) Parallel computation using ipyparallel, which is compatible with Savio BRC-HPC

In [20]:
# create job list
jobs = list(Configuration.select(mesh,"DC.*","RF"))
# evaluate electric potential for each configurations: one electrode at 1V, the rest 0V.
# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
print(jobs)
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


INFO:ipyparallel.cluster.cluster.1667607488-3r2x:Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

KeyboardInterrupt: 

## (4) View result in 3D

This part can be done in other files as well. You only need to define prefix and make sure the two result files for each electrode, i.e. ```<prefix>_<electrode_name>.vtk``` and ```<prefix>_<electrode_name>_mesh.vtk```, are in the same directory as the code. Also don't remember to put the base mesh ```<prefix>_mesh.vtk``` in the same folder.

Result view inlcudes charge distribution, potential/field_square contours.

In [21]:
# base file name for outputs and inputs is the script name
prefix = "htrap_overhang_v2"

view exported base mesh vtk file

In [22]:
Result.view(prefix, '') # don't add anything between '' for mesh view

view simulation result of an electrode

In [23]:
Result.view(prefix, 'RF') # add electrode name between '' for result view

ValueError: Empty meshes cannot be plotted. Input mesh has zero points.

# `IV. Load result for analysis`

All analysis are intended to be done outside of the simulation file. Here shows how to access the simulated data, such as potential (DC and RF), field (RF only), field square (RF only), for each electrode.

In [ ]:
rf_result = Result.from_vtk(prefix, 'RF')
rf_result.grid.to_xyz() # access the (x, y, z) coordinates
rf_result.potential # access the simulated potential data
rf_result.field # access the simulated field data
rf_result.field_square # access the simulated field_square data
print(np.shape(rf_result.field_square))

(21, 21, 21)
